In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath("../.."))
from dotenv import load_dotenv

from examples.config import settings

load_dotenv(".env", override=True)
%load_ext autoreload
%autoreload 2

# Requirements Agent

This notebook demonstrates a requirements engineering agent using the `deepagents` package.

The agent helps with:
1. **Creating Epics**: High-level business capabilities
2. **Creating User Stories**: Detailed requirements linked to epics
3. **Managing Requirements**: Listing, refining, and tracking
4. **Stakeholder Interviews**: Gathering requirements through questions
5. **Generating Summaries**: Overview reports with metrics

**Deepagent Features Demonstrated:**
- Custom domain-specific tools (epics, stories, refinement)
- **Subagent delegation** for specialized stakeholder analysis
- Native filesystem tools for additional documentation
- Native planning tools for complex requirements sessions

## Import the Tools

Let's import our requirements engineering tools.

In [ ]:
from examples.requirements_agent.tools.tools import (
    create_epic,
    create_user_story,
    generate_requirements_summary,
    interview_stakeholder,
    list_requirements,
    refine_requirement,
)

tools = [
    create_epic,
    create_user_story,
    list_requirements,
    refine_requirement,
    generate_requirements_summary,
    interview_stakeholder,
]

for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"Description: {tool.description.split(chr(10))[0]}")
    print()

Tool: create_epic
Description: Create a new Epic requirement document following agile methodology. An Epic is a large body of work that can be broken down into smaller

Tool: create_user_story
Description: Create a user story linked to an existing epic. User stories follow the format: "As a [persona], I want [feature],

Tool: list_requirements
Description: List existing requirements (epics and/or user stories). Use this to see what requirements have already been created and

Tool: refine_requirement
Description: Update or refine an existing epic or user story. Use this to modify requirements based on stakeholder feedback or

Tool: generate_requirements_summary
Description: Generate a comprehensive summary report of all requirements. Creates an overview document showing all epics, their stories,

Tool: interview_stakeholder
Description: Record a stakeholder interview question for requirements elicitation. Use this tool to guide an interactive requirements gathering session.



## Agent Instructions

The system prompt guides the agent's requirements engineering workflow.

In [3]:
from examples.requirements_agent.prompts.prompts import REQUIREMENTS_AGENT_INSTRUCTIONS
from examples.utils import show_prompt

show_prompt(REQUIREMENTS_AGENT_INSTRUCTIONS)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You are a senior Business Analyst and Requirements Engineer specializing in                                    │
│  IT, BI (Business Intelligence), and AI project requirements. You help teams capture, structure,                │
│  and refine requirements using agile methodology.                                                               │
│                                                                                                                 │
│  ## Your Expertise                                                                                              │
│                                                                                                                 │
│  - Translating business needs into structured requirements                                                      │
│  - Agile/Scrum methodology (Epics, User Stories, Acceptance Criteria)                                           │
│  - IT/BI/AI project domain knowledge                                                                            │
│  - Requirements elicitation and stakeholder interviewing                                                        │
│  - Creating clear, testable acceptance criteria                                                                 │
│                                                                                                                 │
│  ## Available Tools                                                                                             │
│                                                                                                                 │
│  ### Requirements Management Tools                                                                              │
│  1. **create_epic**: Create high-level epic requirements                                                        │
│  2. **create_user_story**: Create detailed user stories linked to epics                                         │
│  3. **list_requirements**: View existing requirements                                                           │
│  4. **refine_requirement**: Update requirements based on feedback                                               │
│  5. **generate_requirements_summary**: Create overview reports                                                  │
│  6. **interview_stakeholder**: Ask clarifying questions                                                         │
│                                                                                                                 │
│  ### Native Deepagent Tools (automatically available)                                                           │
│  7. **write_file**: Create additional documentation (meeting notes, glossary, etc.)                             │
│  8. **read_file**: Read previously saved documents                                                              │
│  9. **write_todos**: Plan complex requirements gathering sessions                                               │
│  10. **task**: Delegate to specialist sub-agents for focused analysis                                           │
│                                                                                                                 │
│  ### Sub-Agents Available                                                                                       │
│  - **stakeholder-analyst**: Delegate deep-dive stakeholder interviews for complex domains                       │
│                                                                                                                 │
│  ## Workflow                                                                                                    │
│                                                       

## Creating the Agent

Create the requirements agent with all six tools.

In [ ]:
from deepagents import create_deep_agent
from langchain_google_genai import ChatGoogleGenerativeAI

from examples.requirements_agent.prompts.prompts import (
    REQUIREMENTS_AGENT_INSTRUCTIONS,
    STAKEHOLDER_ANALYST_INSTRUCTIONS,
)

model = ChatGoogleGenerativeAI(model=settings.GOOGLE_MODEL_NAME, temperature=settings.TEMPERATURE)

# Create a stakeholder analyst sub-agent for deep-dive interviews
stakeholder_analyst_subagent = {
    "name": "stakeholder-analyst",
    "model": model,
    "description": "Delegate complex stakeholder interviews to this analyst. Use for deep-dive requirements elicitation on specific domains or when exploring unfamiliar business areas.",
    "system_prompt": STAKEHOLDER_ANALYST_INSTRUCTIONS,
    "tools": [interview_stakeholder],
}

# Create the agent with tools and subagent
agent = create_deep_agent(
    model=model,
    tools=tools,
    system_prompt=REQUIREMENTS_AGENT_INSTRUCTIONS,
    subagents=[stakeholder_analyst_subagent],
)

print("Requirements Agent created successfully!")
print(f"  - Custom tools: {len(tools)}")
print("  - Subagents: stakeholder-analyst")

## Visualize the Agent Graph

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))

## Clean Up Previous Demo Files

Clear any previous demo requirements to start fresh.

In [ ]:
import shutil
from pathlib import Path

output_dir = Path("output")
if output_dir.exists():
    for f in output_dir.glob("*.md"):
        f.unlink()
    for f in output_dir.glob("*.json"):
        f.unlink()
    print("Previous demo files cleaned up.")
else:
    print("No previous files to clean.")

## Test the Agent

Let's test the agent with various requirements engineering scenarios.

In [ ]:
from examples.utils import format_messages

# Test 1: Create an Epic for a BI Dashboard
print("=" * 80)
print("TEST 1: Create an epic for a customer analytics dashboard")
print("=" * 80)

result1 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Create an epic for a Customer Analytics Dashboard that will help our sales team understand customer behavior and identify upsell opportunities.",
            }
        ],
    },
)
format_messages(result1["messages"])

In [ ]:
# Test 2: Create User Stories for the Epic
print("\n" + "=" * 80)
print("TEST 2: Create user stories for the dashboard epic")
print("=" * 80)

result2 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Create 2 user stories for EPIC-001: one for viewing customer segments and one for filtering by date range.",
            }
        ],
    },
)
format_messages(result2["messages"])

In [ ]:
# Test 3: List all requirements
print("\n" + "=" * 80)
print("TEST 3: List all current requirements")
print("=" * 80)

result3 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Show me all the requirements we've created so far.",
            }
        ],
    },
)
format_messages(result3["messages"])

In [ ]:
# Test 4: Refine a requirement
print("\n" + "=" * 80)
print("TEST 4: Update story priority and add story points")
print("=" * 80)

result4 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Update STORY-001 to High priority with 5 story points, and add a note that this is critical for Q1 launch.",
            }
        ],
    },
)
format_messages(result4["messages"])

In [ ]:
# Test 5: Generate summary report
print("\n" + "=" * 80)
print("TEST 5: Generate requirements summary")
print("=" * 80)

result5 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Generate a summary report of all our requirements.",
            }
        ],
    },
)
format_messages(result5["messages"])

In [ ]:
# Test 6: Subagent Delegation - Stakeholder Analysis
print("\n" + "=" * 80)
print("TEST 6: Delegate stakeholder analysis to the specialist subagent")
print("=" * 80)

result6 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I need to understand requirements for a fraud detection ML system. Please delegate to your stakeholder analyst to conduct a focused interview about the data requirements and model performance expectations.",
            }
        ],
    },
)
format_messages(result6["messages"])

## View Generated Files

Let's see the files that were generated.

In [ ]:
output_dir = Path("output")
print("Generated requirement files:")
print("=" * 40)
for f in sorted(output_dir.glob("*.md")):
    print(f"- {f.name}")

In [ ]:
# Display the epic file
epic_files = list(output_dir.glob("EPIC-*.md"))
if epic_files:
    print(f"\nContents of {epic_files[0].name}:")
    print("=" * 40)
    print(epic_files[0].read_text())

## Summary

This requirements agent demonstrates:
1. **Epic creation** - High-level business capabilities with acceptance criteria
2. **User story creation** - Detailed requirements following "As a... I want... So that..." format
3. **Requirements management** - Listing, filtering, and refining requirements
4. **Traceability** - Stories linked to epics with unique IDs
5. **Reporting** - Summary reports with metrics and status tracking
6. **Subagent delegation** - Delegating stakeholder interviews to specialist analyst

**Deepagent Features Used:**
- `create_deep_agent()` with custom tools and subagents
- Native tools (write_file, write_todos) available for documentation and planning
- Context isolation via subagents for focused stakeholder analysis

The agent is designed for IT, BI, and AI projects, with domain-specific guidance in the system prompt.